In [1]:


from dotenv import load_dotenv
load_dotenv()
import rag_helpers as rh

ctx = rh.get_rag_helper_context()


weaviate client connected: True


In [3]:

query = "How do I get in touch with support?"

similar_texts = ctx.meta_query(query)

print(f"\nRetrieved {len(similar_texts)} contexts for query '{query}':")
for i, res in enumerate(similar_texts, 1):
    print(f"\n{i}. Distance: {res.distance}")
    print(f"Text: {res.text}\n\n")



Retrieved 3 contexts for query 'How do I get in touch with support?':

1. Distance: 0.6159154176712036
Text: Technical Support

If you need further assistance, contact the Tesla Support team via the Contact Us page:

tesla.com/support/energy/more/additional-support/contact-us

Have the following information available when contacting Tesla:

- Owner name
- Best way for Tesla to contact you (name, phone number, email)
- Powershare Gateway serial number
- Brief description of the issue

Charging and Energy Consumption                                187
---



2. Distance: 0.6914113759994507
Text: Contacting Tesla

For detailed information about your Cybertruck, go to http://www.tesla.com and log on to your Tesla account or sign up to get an account.

If you have any questions or concerns about your Cybertruck, in the United States, Canada or Puerto Rico, call 1-877-79TESLA (1-877-798-3752) and in Mexico, call 1-800-228-8145.

NOTE: You can also use voice commands to provide feedback to T

In [4]:
answer = ctx.chat_response(query)
print("----------------------------------\n\n")
print(f"Question: {query}\n\nAnswer: {answer}")

prompt: Answer the question using ONLY the information provided in the context below. 
Do not add any general knowledge or information not contained in the context."

Context:
Technical Support

If you need further assistance, contact the Tesla Support team via the Contact Us page:

tesla.com/support/energy/more/additional-support/contact-us

Have the following information available when contacting Tesla:

- Owner name
- Best way for Tesla to contact you (name, phone number, email)
- Powershare Gateway serial number
- Brief description of the issue

Charging and Energy Consumption                                187
---

---

Contacting Tesla

For detailed information about your Cybertruck, go to http://www.tesla.com and log on to your Tesla account or sign up to get an account.

If you have any questions or concerns about your Cybertruck, in the United States, Canada or Puerto Rico, call 1-877-79TESLA (1-877-798-3752) and in Mexico, call 1-800-228-8145.

NOTE: You can also use voice co